In [ ]:
!pip install faiss-cpu sentence-transformers langid transformers torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.9 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np
import pandas as pd
import langid
from sentence_transformers import SentenceTransformer
from transformers import MarianMTModel,MarianTokenizer,pipeline


In [ ]:
faq_data = {
    "question": [
        "What is the last date for fee submission?",
        "How to apply for a scholarship?",
        "Where can I find the timetable?",
        "Who should I contact for hostel issues?",
        "How to get my ID card reissued?"
    ],
    "answer": [
        "The last date for fee submission is 30th September.",
        "You can apply for a scholarship via the student portal under 'Scholarship Section'.",
        "The timetable is available on the official college website.",
        "For hostel issues, please contact the warden in the hostel office.",
        "For ID card reissue, visit the admin office with a request form and ID proof."
    ]
}
faq_df = pd.DataFrame(faq_data)


In [ ]:
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
generator = pipeline("text-generation", model="distilgpt2", max_new_tokens=30)


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
src_lang = "Helsinki-NLP/opus-mt-hi-en"
tgt_lang = "Helsinki-NLP/opus-mt-en-hi"



In [ ]:
trans_tokenizer_src = MarianTokenizer.from_pretrained(src_lang)
trans_model_src = MarianMTModel.from_pretrained(src_lang)

trans_tokenizer_tgt = MarianTokenizer.from_pretrained(tgt_lang)
trans_model_tgt = MarianMTModel.from_pretrained(tgt_lang)



tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def translate(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)


In [ ]:
corpus = faq_df["question"].tolist()
corpus_embeddings = embedder.encode(corpus)
dim = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(corpus_embeddings, dtype="float32"))


In [ ]:
def chatbot(query, k=1):
    lang, conf = langid.classify(query)
    print(f"[LangID] Detected: {lang} (conf={conf:.2f})")

    if lang != "en":
        query = translate(query, trans_tokenizer_src, trans_model_src)
        print(f"[Translation] To English: {query}")

    query_vec = embedder.encode([query])
    distances, indices = index.search(np.array(query_vec, dtype="float32"), k)
    base_answer = faq_df.iloc[indices[0][0]]["answer"]

    gen_input = f"Q: {query}\nA: {base_answer}\nBetter Answer:"
    gen_answer = generator(gen_input, num_return_sequences=1)[0]["generated_text"]

    return gen_answer


In [ ]:
queries = [
    "When is the deadline to pay fees?",

    "I lost my ID card, what should I do?"
]

for q in queries:
    print(f"\n👩‍🎓 User: {q}")
    print(f"🤖 Bot:  {chatbot(q)}\n")



👩‍🎓 User: When is the deadline to pay fees?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[LangID] Detected: en (conf=-100.26)
🤖 Bot:  Q: When is the deadline to pay fees?
A: The last date for fee submission is 30th September.
Better Answer:
If you have more details on fees that are needed to complete your online application, please look at this post for more details, including how to use


👩‍🎓 User: मुझे छात्रवृत्ति कैसे मिलेगी?
[LangID] Detected: hi (conf=-171.09)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[Translation] To English: How would I get studentship?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🤖 Bot:  Q: How would I get studentship?
A: You can apply for a scholarship via the student portal under 'Scholarship Section'.
Better Answer: The student portal will allow you to apply for the scholarship via the student portal under 'Scholarship Section'.
If you want to apply for a


👩‍🎓 User: I lost my ID card, what should I do?
[LangID] Detected: en (conf=-63.87)
🤖 Bot:  Q: I lost my ID card, what should I do?
A: For ID card reissue, visit the admin office with a request form and ID proof.
Better Answer:
You must answer the following questions.
1. Is the ID card number your ID card is?
2. Is it your ID card number



In [ ]:
queries = [
    "When is the deadline to pay fees?",

    "I lost my ID card, what should I do?"
]